In [ ]:
#@markdown <center><img src="https://i.imgur.com/Vz0uo7C.png" width="275">
REMOTE           = "" #@param {type:"string"} 
API_KEY          = "c42fc7df" #@param {type:"string"} 
HIDE_REACT_SCORE = 10 #@param {type:"integer"}
HIDE_POST_COUNT  = 25 #@param {type:"integer"}
 
import base64, os, regex, requests, subprocess, urllib.request
from IPython.display import clear_output
urllib.request.urlretrieve("https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py", os.path.expanduser("~") + "/.ipython/ttmg.py")
from ttmg import loadingAn, textAn
!rm -rf "/content/sample_data"
loadingAn()
textAn("Installing Dependencies...", ty='twg')
!wget "https://mediaarea.net/repo/deb/repo-mediaarea_1.0-13_all.deb" -q && dpkg -i "repo-mediaarea_1.0-13_all.deb" > /dev/null 2>&1 && rm -f "repo-mediaarea_1.0-13_all.deb"
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y > /dev/null 2>&1 && apt install ffmpeg mediainfo &>/dev/null
!bash <(wget -qO- https://rclone.org/install.sh) &>/dev/null
clear_output()
!echo "Upload the rClone.conf"
from google.colab import files
uploaded = files.upload()
clear_output()
!mkdir --parents "/root/.config/rclone/" && mv "/content/rclone.conf" "/root/.config/rclone/" && mkdir --parents "/content/{REMOTE}/"
!rclone mount "{REMOTE}": "/content/{REMOTE}/" --buffer-size 32M --daemon --max-read-ahead 32M --poll-interval 30s --read-only --use-mmap --vfs-cache-mode full --vfs-read-chunk-size-limit 16M --vfs-read-chunk-size 8M

In [ ]:
#@title <b>Movie and TV Shows Template
def BlackPearl_Posting_Template(PATH, FOLDER, SCREENSHOTS, TONEMAPPING, IMDB, YOUTUBE):
  removeTemp('./.nfo')
  TEMP = subprocess.call(['mediainfo', '--logfile=./.nfo', PATH], stdout=open(os.devnull, 'wb'))
  LINK = GenerateLink(PATH, FOLDER)
  if SCREENSHOTS == True:
    SCREENSHOTS = "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER]"
    TONEMAPPING = 'scale=min(1920\,iw):-1,zscale=transfer=linear,tonemap=hable,zscale=transfer=bt709' if TONEMAPPING == True else 'scale=min(1920\,iw):-1'
    SCREENSHOTS += CreateScreenshots(PATH, "02:30.00", TONEMAPPING)
    SCREENSHOTS += CreateScreenshots(PATH, "05:00.00", TONEMAPPING)
    SCREENSHOTS += CreateScreenshots(PATH, "07:30.00", TONEMAPPING)
    SCREENSHOTS += "[/CENTER][/SPOILER]"
    if SCREENSHOTS == "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER][/CENTER][/SPOILER]": SCREENSHOTS = "" 
  else: SCREENSHOTS = ""
  IMDB = GetIMDb(PATH, IMDB)
  YOUTUBE = GetYouTube(YOUTUBE, IMDB)
  BBCODE = "[CENTER][URL='{}'][IMG WIDTH='350px']{}[/IMG][/URL][/CENTER]\n".format(regex.sub("@\..*.jpg","@.png", IMDB['Poster']), regex.sub("@\..*.jpg","@.png", IMDB['Poster'])) if IMDB['Poster'] != "N/A" else ""
  BBCODE += "[CENTER][URL='https://blackpearl.biz/search/1/?q={}&o=date'][FORUMCOLOR][B][SIZE=6]{} ({})[/SIZE][/B][/FORUMCOLOR][/URL][/CENTER]\n".format(IMDB['imdbID'], IMDB['Title'], IMDB['Year'])
  BBCODE += "[CENTER][URL='https://imdb.com/title/{}'][IMG WIDTH='46px']https://ia.media-imdb.com/images/M/MV5BMTk3ODA4Mjc0NF5BMl5BcG5nXkFtZTgwNDc1MzQ2OTE@.png[/IMG][/URL][/CENTER]".format(IMDB['imdbID'])
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Plot Summary :[/B][/FORUMCOLOR][/SIZE]\n\n[JUSTIFY]{}[/JUSTIFY][/INDENT]".format(IMDB['Plot']) if IMDB['Plot'] != "N/A" else ""
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]IMDb Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[LIST]"
  BBCODE += "[*][FORUMCOLOR][B]Rating :[/B][/FORUMCOLOR] {}\n".format(IMDB['Rated']) if IMDB['Rated'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Genre :[/B][/FORUMCOLOR] {}\n".format(IMDB['Genre']) if IMDB['Genre'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Directed By :[/B][/FORUMCOLOR] {}\n".format(IMDB['Director']) if IMDB['Director'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Written By :[/B][/FORUMCOLOR] {}\n".format(IMDB['Writer']) if IMDB['Writer'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Starring :[/B][/FORUMCOLOR] {}\n".format(IMDB['Actors']) if IMDB['Actors'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Release Date :[/B][/FORUMCOLOR] {}\n".format(IMDB['Released']) if IMDB['Released'] != "N/A" else ""
  BBCODE += "[*][FORUMCOLOR][B]Awards :[/B][/FORUMCOLOR] {}".format(IMDB['Awards']) if IMDB['Awards'] != "N/A" else ""
  BBCODE += "[/LIST][HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Official Trailer :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n\n[CENTER][MEDIA=YouTube]{}[/MEDIA][/CENTER]".format(YOUTUBE)
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]"
  MEDIAINFO = GetMediaInfo(PATH)
  BBCODE += "[CODE TITLE='Media Info']{}[/CODE]\n".format(MEDIAINFO)
  BBCODE += SCREENSHOTS
  BBCODE += "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Download Link :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[CENTER]{}[/CENTER]".format(LINK)
  return BBCODE
 
def removeTemp(PATH):
  try: os.remove(PATH)
  except: pass
 
def GenerateLink(PATH, FOLDER):
  LINK = PATH.replace('/content/'+REMOTE+'/', '')
  if FOLDER == "PARENT FOLDER": LINK = os.path.dirname(LINK)
  if FOLDER == "GRANDPARENT FOLDER": LINK = os.path.dirname(os.path.dirname(LINK))
  LINK = !rclone link "{REMOTE}":"{LINK}"
  LINK = "[HIDEREACT=1,2,3,4,5,6,7,8][DOWNCLOUD]" + LINK[0] + "[/DOWNCLOUD][/HIDEREACT]"
  if HIDE_REACT_SCORE != 0: LINK = "[HIDEREACTSCORE={}]{}[/HIDEREACTSCORE]".format(HIDE_REACT_SCORE, LINK)
  if HIDE_POST_COUNT != 0: LINK = "[HIDEPOSTS={}]{}[/HIDEPOSTS]".format(HIDE_POST_COUNT, LINK)
  return LINK 
  
def CreateScreenshots(PATH, TIME, TONEMAPPING):
  removeTemp('./frame.png')
  TEMP = subprocess.run(['ffmpeg', '-hide_banner', '-loglevel', 'error', '-ss', TIME, '-i', PATH, '-vf', TONEMAPPING, '-vframes', '1', '-q:v', '2', '-y', './frame.png'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout.decode()
  SCREENSHOT_LINK = requests.post("https://api.imgbb.com/1/upload",{"key":"6b1c29d7993f76b154e2c704f1d13879","image":base64.b64encode(open("./frame.png", "rb").read())})
  SCREENSHOT_LINK = "[URL='{}'][IMG WIDTH='800px']{}[/IMG][/URL]\n".format(SCREENSHOT_LINK.json()['data']['url'], SCREENSHOT_LINK.json()['data']['url']) if str(SCREENSHOT_LINK.status_code) == "200" else ""
  removeTemp('./frame.png')
  return SCREENSHOT_LINK 
 
def GetIMDb(PATH, IMDB):
  if IMDB.lower() == "none" or IMDB == "":
    try: YEAR = "&y="+regex.findall(r"(19\d{2}|20\d{2})", PATH)[-1]
    except: YEAR = "" 
    TITLE = regex.sub('[^a-zA-Z0-9]', '+', os.path.basename(PATH)).split(f"+{YEAR.replace('&y=','')}+")[0].split("+S0")[0].split("+S1")[0].split("+S2")[0].split("+UHD")[0].split("+1080p")[0].split("+2160p")[0].split("+HYBRID")[0].split("+EXTENDED")[0]
    IMDB = requests.get(f'http://www.omdbapi.com/?t={TITLE}{YEAR}&apikey={API_KEY}&r=json').json()
  else: IMDB = requests.get(f'http://www.omdbapi.com/?i={regex.search(r"(tt[0-9]{7,8})", IMDB)[1]}&apikey={API_KEY}&r=json').json()
  return IMDB
 
def GetYouTube(YOUTUBE, IMDB):
  YOUTUBE = regex.search(r"watch\?v=(\S{11})", str(requests.get("https://www.youtube.com/results?search_query="+(regex.sub('[^a-zA-Z0-9]', '+',IMDB['Title']+"+"+(IMDB['Year'][0:4])))+"+Official+Trailer").content))[1] if (YOUTUBE.lower() == "none" or YOUTUBE == "") else regex.findall(r"([A-Za-z0-9-_]{11})", YOUTUBE)[-1] 
  return YOUTUBE
 
def GetMediaInfo(PATH):
  NFO = open('./.nfo', 'r', encoding='utf-8').readlines(); NFO.pop(); del NFO[1]; NFO[1] = "Complete Name                            : {}\n".format(os.path.basename(PATH))
  removeTemp('./.nfo')
  return ''.join(NFO)

In [ ]:
IMDB             = "" #@param {type:"string"}
YOUTUBE          = "" #@param {type:"string"}
PATH             = "" #@param {type:"string"}
FOLDER           = "Disabled" #@param ["Disabled", "PARENT FOLDER", "GRANDPARENT FOLDER"]
SCREENSHOTS      = False #@param {type:"boolean"}
TONEMAPPING      = False #@param {type:"boolean"}
print(BlackPearl_Posting_Template(PATH, FOLDER, SCREENSHOTS, TONEMAPPING, IMDB, YOUTUBE))